In [1]:
import os
import pickle
import random
import numpy as np
import pandas as pd
import tkinter as tk
import tensorflow as tf
from pathlib import Path
from tkinter import messagebox
from PIL import Image, ImageTk
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
def train_model(data_path: str = './data', epochs: int = 100):
    # Define the paths
    train_dir = f'{data_path}/train'
    valid_dir = f'{data_path}/valid'
    test_dir = f'{data_path}/test'
    
    # Load the CSV file
    dataset_csv = pd.read_csv(f'{data_path}/cards.csv')

    # Create ImageDataGenerator for loading and augmenting images
    train_datagen = ImageDataGenerator(rescale=1./255)
    valid_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )
    
    valid_generator = valid_datagen.flow_from_directory(
        valid_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    model_path = f'./models/{epochs}_model.h5'
    if Path(model_path).exists():
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(512, activation='relu'),
            Dropout(0.5),
            Dense(53, activation='softmax')  # 53 classes for 53 cards
        ])
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        model.load_weights(model_path)  # Load your trained model weights
        return model
    else:
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(512, activation='relu'),
            Dropout(0.5),
            Dense(53, activation='softmax')  # 53 classes for 53 cards
        ])
        
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        
        # Train the model
        history = model.fit(
            train_generator,
            epochs=epochs,
        )
    
        # Evaluate the model
        loss, accuracy = model.evaluate(test_generator)
        print(f'Test accuracy: {accuracy}')
    
        model.save(model_path)

    return model

In [3]:
model = train_model()

Found 7624 images belonging to 53 classes.
Found 265 images belonging to 53 classes.
Found 265 images belonging to 53 classes.
Epoch 1/100


C:\Python\Python3_11_7\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
C:\Python\Python3_11_7\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


239/239 ━━━━━━━━━━━━━━━━━━━━ 205s 856ms/step - accuracy: 0.1619 - loss: 3.9119
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 206s 859ms/step - accuracy: 0.5323 - loss: 1.7012
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 206s 863ms/step - accuracy: 0.7349 - loss: 0.9802
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 207s 864ms/step - accuracy: 0.8521 - loss: 0.5584
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 204s 855ms/step - accuracy: 0.9151 - loss: 0.3250
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 206s 862ms/step - accuracy: 0.9477 - loss: 0.2084
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 198s 827ms/step - accuracy: 0.9571 - loss: 0.1746
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 195s 817ms/step - accuracy: 0.9674 - loss: 0.1424
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 196s 821ms/step - accuracy: 0.9669 - loss: 0.1191
Epoch 10/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 195s 816ms/step - accuracy: 0.9812 - loss: 0.0794
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 195s 815ms/step - accuracy: 0.9777 - loss: 0.0978
Epoch 12/

Test accuracy: 0.8113207817077637


In [4]:
class CardCounter:
    high_cards = ('ten', 'jack', 'queen', 'king', 'ace')
    low_cards = ('two', 'three', 'four', 'five', 'six')

    def __init__(self, model, n_decks: int = 8):
        self.model = model
        self.total_cards = 52 * n_decks
        
        self._remaining_cards = {
            'total': self.total_cards, 
            'high': int(round(self.total_cards * 5 / 13)), 
            'low': int(round(self.total_cards * 5 / 13)),
            'neutral': int(round(self.total_cards * 3 / 13))
        }
        
        self._count = 0

        self.test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
            './data/test',
            target_size=(224, 224),
            batch_size=32,
            class_mode='categorical'
        )
        
        self.card_counts = {label: 0 for label in self.test_generator.class_indices.keys()}

    @property
    def count(self):
        return self._count

    @count.setter
    def count(self, count: int):
        self._count = count

    @property
    def remaining_cards(self):
        return self._remaining_cards

    @remaining_cards.setter
    def remaining_cards(self, remaining_cards: int):
        self._remaining_cards = remaining_cards

    def classify_card(self, image):
        image = tf.image.resize(image, (224, 224))
        image = tf.expand_dims(image, 0)  # Add batch dimension
        predictions = self.model.predict(image)
        class_index = tf.argmax(predictions[0]).numpy()
        class_label = list(self.test_generator.class_indices.keys())[class_index]
        print(class_label)
        return class_label
    
    def count_cards(self, images):
        high_cards = 0
        low_cards = 0
        neutral_cards = 0
        card_labels = []
        for image in images:
            card_label = self.classify_card(image=image)
            rank = card_label.split()[0]
            suit = card_label.split()[-1]
            card_labels.append((rank, suit))
            self.card_counts[card_label] += 1
            
            if card_label.split()[0] in self.high_cards:
                self.count -= 1
                high_cards += 1
            elif card_label.split()[0] in self.low_cards:
                self.count += 1
                low_cards += 1
            else:
                neutral_cards += 1
    
        # Calculate remaining cards
        self.remaining_cards['total'] -= len(images)
        self.remaining_cards['high'] -= high_cards
        self.remaining_cards['low'] -= low_cards
        self.remaining_cards['neutral'] -= neutral_cards
    
        prob_high = self.remaining_cards['high'] / self.remaining_cards['total'] if self.remaining_cards['total'] > 0 else 0
        prob_low = self.remaining_cards['low'] / self.remaining_cards['total'] if self.remaining_cards['total'] > 0 else 0
        prob_neutral = self.remaining_cards['neutral'] / self.remaining_cards['total'] if self.remaining_cards['total'] > 0 else 0

        return prob_high, prob_low, prob_neutral, card_labels

In [5]:
image_paths = [r'.\data\test\ace of clubs\1.jpg', r'.\data\test\nine of clubs\1.jpg', 
               r'.\data\test\six of clubs\1.jpg', r'.\data\test\king of hearts\1.jpg']

images = [tf.io.read_file(image_path) for image_path in image_paths]
images = [tf.image.decode_jpeg(image, channels=3) for image in images]

In [6]:
card_counter = CardCounter(model=model)
prob_high, prob_low, prob_neutral, card_labels = card_counter.count_cards(images=images)
print(f'Probability of next card being +1 count: {prob_low:.2f}')
print(f'Probability of next card being -1 count: {prob_high:.2f}')
print(f'Probability of next card being 0 count: {prob_neutral:.2f}')

Found 265 images belonging to 53 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
four of clubs
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
nine of spades
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
king of diamonds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
king of hearts
Probability of next card being +1 count: 0.39
Probability of next card being -1 count: 0.38
Probability of next card being 0 count: 0.23


In [7]:
class BlackjackGame:
    def __init__(self, root, bankroll: int, model_path: str):
        self.root = root
        self.root.title("Blackjack Game")

        self.suits = ['hearts', 'diamonds', 'clubs', 'spades']
        self.ranks = {
            'two': 2, 
            'three': 3, 
            'four': 4, 
            'five': 5, 
            'six': 6, 
            'seven': 7, 
            'eight': 8, 
            'nine': 9, 
            'ten': 10, 
            'jack': 10, 
            'queen': 10, 
            'king': 10, 
            'ace': 11
        }
        
        self.model_path = model_path

        # Load the trained model
        self.model = self.load_model()

        # Initialize game variables
        self.deck = self.create_deck()
        self.player_hand = []
        self.dealer_hand = []
        self.card_images = []
        self.wins = 0
        self.losses = 0
        self.draws = 0
        self.bankroll = bankroll

        self.card_counter = CardCounter(model=model)

        # Create GUI elements
        self.create_widgets()

    def load_model(self):
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(512, activation='relu'),
            Dropout(0.5),
            Dense(53, activation='softmax')  # 53 classes for 53 cards
        ])
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        model.load_weights(self.model_path)  # Load your trained model weights
        return model

    def create_deck(self):
        deck = [(rank, suit) for rank in self.ranks.keys() for suit in self.suits]
        random.shuffle(deck)
        return deck

    def create_widgets(self):
        self.player_label = tk.Label(self.root, text="Player's Hand")
        self.player_label.pack()
        self.player_frame = tk.Frame(self.root)
        self.player_frame.pack()

        self.dealer_label = tk.Label(self.root, text="Dealer's Hand")
        self.dealer_label.pack()
        self.dealer_frame = tk.Frame(self.root)
        self.dealer_frame.pack()

        self.hit_button = tk.Button(self.root, text="Hit", command=self.hit)
        self.hit_button.pack(side=tk.LEFT)
        self.stand_button = tk.Button(self.root, text="Stand", command=self.stand)
        self.stand_button.pack(side=tk.LEFT)
        self.reset_button = tk.Button(self.root, text="Reset", command=self.reset_game)
        self.reset_button.pack(side=tk.LEFT)

        self.prob_label = tk.Label(self.root, text="Probabilities - High: 0.00, Low: 0.00, Neutral: 0.00")
        self.prob_label.pack()

        self.game_label = tk.Label(self.root, text="Game 1: Wins - 0, Losses - 0, Draws - 0 -- Cards Left = 416")
        self.game_label.pack()

        self.wallet_label = tk.Label(self.root, text=f"Bankroll: ${self.bankroll}")
        self.wallet_label.pack()

    def start_game(self):
        self.deal_card_to_player()
        self.deal_card_to_dealer()
        self.deal_card_to_player()
        self.deal_card_to_dealer()
        
        player_value = self.calculate_hand_value(self.player_hand)
        dealer_value = self.calculate_hand_value(self.dealer_hand)
        
        if player_value == 21 and dealer_value == 21:
            self.draws += 1
            self.update_game_label(0)
            messagebox.showinfo("Blackjack", "Draw!")
            self.reset_game()
        elif player_value == 21:
            self.wins += 1
            self.update_game_label(100)
            messagebox.showinfo("Blackjack", "Player wins!")
            self.reset_game()
        elif dealer_value == 21:
            self.losses += 1
            self.update_game_label(-100)
            messagebox.showinfo("Blackjack", "Dealer wins!")
            self.reset_game()
        
    def update_probabilities_label(self, prob_high, prob_low, prob_neutral):
        self.prob_label.config(text=f'Probabilities - High: {prob_high:.2f}, Low: {prob_low:.2f}, Neutral: {prob_neutral:.2f}')
    
    def update_game_label(self, rewards: int):
        self.game_label.config(text=f'Game {self.wins + self.losses + self.draws}: Wins - {self.wins}, Losses - {self.losses}, Draws - {self.draws} -- Cards Left = {self.card_counter.remaining_cards["total"]}')
        self.bankroll += rewards
        self.wallet_label.config(text=f"Bankroll: ${self.bankroll}")
    
    def deal_card_to_player(self):
        card = self.deck.pop()
        self.player_hand.append(card)
        img_path = f'./data/test/{card[0]} of {card[1]}/1.jpg'
        
        # Debugging: Check if the image path exists
        if not os.path.exists(img_path):
            print(f"Image path does not exist: {img_path}")
            return
        
        images = [tf.image.decode_jpeg(tf.io.read_file(img_path), channels=3)]
        prob_high, prob_low, prob_neutral, predicted_cards = self.card_counter.count_cards(images=images)

        try:
            img = Image.open(img_path)
            img = img.resize((100, 150), Image.LANCZOS)
            photo = ImageTk.PhotoImage(img)
            self.card_images.append(photo)  # Keep a reference to the image
            frame = tk.Frame(self.player_frame)
            frame.pack(side=tk.LEFT)
            label = tk.Label(frame, image=photo)
            label.image = photo  # Ensure reference is kept by the label
            label.pack()
            tk.Label(frame, text=f'{card[0]} of {card[1]}').pack()
        except Exception as e:
            print(f"Failed to load image {img_path}: {e}")

        # Update probabilities label
        self.update_probabilities_label(prob_high, prob_low, prob_neutral)
    
    def deal_card_to_dealer(self):
        real_card = self.deck.pop()
        real_img_path = f'./data/test/{real_card[0]} of {real_card[1]}/1.jpg'
        
        # Debugging: Check if the image path exists
        if not os.path.exists(real_img_path):
            print(f"Image path does not exist: {real_img_path}")
            return

        images = [tf.image.decode_jpeg(tf.io.read_file(real_img_path), channels=3)]
        prob_high, prob_low, prob_neutral, predicted_cards = self.card_counter.count_cards(images=images)
        predicted_card = predicted_cards[0]
        predicted_img_path = f'./data/test/{predicted_card[0]} of {predicted_card[1]}/1.jpg'

        if not os.path.exists(predicted_img_path):
            print(f"Image path does not exist: {predicted_img_path}")
            return
        
        self.dealer_hand.append(real_card)

        try:
            real_img = Image.open(real_img_path)
            real_img = real_img.resize((100, 150), Image.LANCZOS)
            real_photo = ImageTk.PhotoImage(real_img)
            self.card_images.append(real_photo)
            frame = tk.Frame(self.dealer_frame)
            frame.pack(side=tk.LEFT)
            real_label = tk.Label(frame, image=real_photo)
            real_label.image = real_photo  # Ensure reference is kept by the label
            real_label.pack()
            tk.Label(frame, text=f'Actual: {real_card[0]} of {real_card[1]}\nPredicted: {predicted_card[0]} of {predicted_card[1]}').pack()
        except Exception as e:
            print(f"Failed to load image {real_img_path} or {predicted_img_path}: {e}")

        # Update probabilities label
        self.update_probabilities_label(prob_high, prob_low, prob_neutral)

    def hit(self):
        self.deal_card_to_player()
        player_value = self.calculate_hand_value(self.player_hand)
        if player_value > 21:
            self.losses += 1
            self.update_game_label(-100)
            messagebox.showinfo("Blackjack", "Player busts! Dealer wins!")
            self.reset_game()

        if player_value == 21:
            self.wins += 1
            self.update_game_label(100)
            messagebox.showinfo("Blackjack", "Player wins!")
            self.reset_game()

    def stand(self):
        player_value = self.calculate_hand_value(self.player_hand)
        if player_value == 21:
            self.wins += 1
            self.update_game_label(100)
            messagebox.showinfo("Blackjack", "Player wins!")
            self.reset_game()
            
        while self.calculate_hand_value(self.dealer_hand) < 17:
            self.deal_card_to_dealer()
            
        dealer_value = self.calculate_hand_value(self.dealer_hand)
        
        if dealer_value > 21 or player_value > dealer_value:
            self.wins += 1
            self.update_game_label(100)
            messagebox.showinfo("Blackjack", "Player wins!")
            self.reset_game()
        else:
            self.losses += 1
            self.update_game_label(-100)
            messagebox.showinfo("Blackjack", "Dealer wins!")
            self.reset_game()

    def reset_game(self):
        for widget in self.player_frame.winfo_children():
            widget.destroy()
            
        for widget in self.dealer_frame.winfo_children():
            widget.destroy()
            
        self.deck = self.create_deck()
        self.player_hand = []
        self.dealer_hand = []
        self.start_game()
        self.enable_buttons()

    def calculate_hand_value(self, hand):
        value = 0
        ace_count = 0
        for card in hand:
            rank = card[0]
            value += self.ranks[rank]
            
            if rank == 'ace':
                ace_count += 1
                
        while value > 21 and ace_count:
            value -= 10
            ace_count -= 1
            
        return value

    def disable_buttons(self):
        self.hit_button.config(state=tk.DISABLED)
        self.stand_button.config(state=tk.DISABLED)

    def enable_buttons(self):
        self.hit_button.config(state=tk.NORMAL)
        self.stand_button.config(state=tk.NORMAL)

In [8]:
# Create the main window
root = tk.Tk()
game = BlackjackGame(root, 18000, './models/25_model.h5')
game.start_game()
root.mainloop()

Found 265 images belonging to 53 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
two of diamonds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
ten of hearts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
jack of hearts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
five of hearts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
nine of spades
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
eight of hearts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
three of spades
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
two of spades
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
two of diamonds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
ace of diamonds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
five of spades
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
king of diamonds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
four of diamonds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
ten of clubs
